C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\VSP

In [2]:
import pandas as pd
import os
import numpy as np
import datetime as dt

from os import listdir
from os.path import isfile, join
from os.path import getmtime

from zebra import remov_duplicates, human_format
from directories import parametriche,parametriche_files, budest_prodotto_anno, budest_prodotto_anno_files

In [2]:
idx = pd.IndexSlice

In [4]:
pd.set_option('display.max_rows', 100)

# Sources

## Parametriche

In [5]:
par = {}
for file in parametriche_files:
    file_name=file.split(".")[0]
    par[f"{file_name}"] = pd.read_excel(parametriche+"\\"+file)

In [12]:
cod_prod = par["Classificazione_codice_prodotto"]
cod_prod["Codice PRODOTTO"] = cod_prod["Codice PRODOTTO"].apply(lambda x: str(x))
cod_prod = cod_prod.loc[:, ['GTM BDG', 'Solution', 'Sub solution', 'VSP',"Codice PRODOTTO"]]

## Raccolto

In [9]:
dfs = []
for file in budest_prodotto_anno_files:
    data = pd.read_excel(budest_prodotto_anno+"\\"+file, na_values="#", dtype={"Codice ABBONAMENTO":str})
    data["Ultimo Agg."] = dt.date.fromtimestamp(getmtime(budest_prodotto_anno+"\\"+file))
    dfs.append(data)

fatt = pd.concat(dfs, axis=0, ignore_index=True)

In [ ]:
raccolto = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Analisi clienti\Parco clienti\Data export\Budest\Budest prodotto anno\\"

onlyfiles = [f for f in listdir(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Analisi clienti\Parco clienti\Data export\Budest\Budest prodotto anno") if isfile(join(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Analisi clienti\Parco clienti\Data export\Budest\Budest prodotto anno", f))]

dfs = []
for file in onlyfiles:
    data = pd.read_excel(raccolto+file, na_values="#", dtype={"Codice ABBONAMENTO":str})
    data["Ultimo Agg."] = dt.date.fromtimestamp(getmtime(raccolto+file))
    dfs.append(data)

fatt = pd.concat(dfs, axis=0, ignore_index=True)

In [10]:
fatt["Raccolto"] = fatt.loc[:,["Altro","Rinnovi Software","Vendite Editoria","Vendite Software"]].sum(axis=1)
fatt["Vendite"] = fatt.loc[:,["Vendite Editoria","Vendite Software"]].sum(axis=1)
fatt["Rinnovi"] = fatt.loc[:,["Altro","Rinnovi Software"]].sum(axis=1)

fatt["Codice PRODOTTO"] = fatt.loc[:,"Codice PRODOTTO"].apply(lambda x: str(x))

In [13]:
fatt.merge(cod_prod, on="Codice PRODOTTO", how="left").groupby(["Anno Inserimento Ord","VSP","GTM BDG","Solution","Sub solution","Prodotto (SW)", "Sales Director", "Segmento Aggregato"], dropna=False).agg({"Vendite":"sum","Rinnovi":"sum", "Raccolto":"sum"}).reset_index().style.format(thousands=".", precision=0, subset=["Vendite", "Rinnovi", "Raccolto"]).to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\VSP\Fatturato finanziario 2017 - 2022 v3.xlsx", index=False)